## UTILIZANDO **GRADIO** PARA VISUALIZAÇÃO DAS CLASSIFICAÇÕES (SINGLE OU MULTI LABEL)

- **Gradio** é uma ferramenta em Python que simplifica a criação e compartilhamento de demos ou aplicativos web para modelos de ML, APIs ou funções, sem necessidade de experiência em JavaScript, CSS ou hospedagem web.
- Neste _notebook_, abordaremos a criação de uma **interface gráfica** básica utilizando o **Gradio** e como executar um modelo de **classificação de objetos** em conjunto com essa interface.

#### 🔎 Verificar se está sendo utilizado CPU ou GPU

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices()

#
---

### 1º PASSO: IMPORTAR AS BIBLIOTECAS

In [ ]:
import os
import zipfile

import gradio as gr
import numpy as np
import tensorflow as tf

# 
---

### 2º PASSO: CRIAR A INTERFACE RESPONSÁVEL POR EXECUTAR O MODELO

Vamos empregar **variáveis globais** para simplificar o código, porém essa não é a única forma de implementar a função. 
_(Sinta-se à vontade para otimizar o código se desejar)_

In [ ]:
# Diretório onde o modelo está armazenado
DATA_PATH = os.path.join(os.pardir, "data", "models", "single")

# Nome do modelo .tflite a ser utilizado
MODEL_FILE = "ex_raspberry_identifier.tflite"

# Caminho completo do diretório até o arquivo do modelo
MODEL_PATH = os.path.join(DATA_PATH, MODEL_FILE)

#
---

### 3º PASSO: EXTRAIR AS CATEGORIAS DO MODELO QUE SERÁ EXECUTADO

O código abaixo imprime no _output_ quais categorias existem dentro do modelo carregado na variável global **MODEL_PATH**

In [ ]:
with zipfile.ZipFile(MODEL_PATH, "r") as archive:
    for file in archive.namelist():
        LABELS = archive.read(file).decode(encoding="utf-8").split()
        display(LABELS)

O código abaixo extrai o arquivo **dict.txt** do modelo e o salva no mesmo diretório

In [ ]:
with zipfile.ZipFile(MODEL_PATH, "r") as zip_ref:
    zip_ref.extractall(DATA_PATH)

#
---

### 4º PASSO: CRIAR O INTERPRETADOR QUE USARÁ O MODELO E ALOCAR NA MEMÓRIA

In [ ]:
# Criação do interpretador do modelo
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)

# Alocação na memória
interpreter.allocate_tensors()

#
---

### 5º PASSO: CRIAR FUNÇÃO DE PREVISÃO COM O INTERPRETADOR DO MODELO

Esta é a função que irá rodar o modelo usando do interpretador alocado e das variáveis globais

In [ ]:
def predict(image):
    global LABELS
    global interpreter

    # Recolher detalhes sobre os parâmetros de entrada e saída do modelo.
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # Reajustar para o tamanho 224x224
    image = image.resize((224, 224))

    # Imagem será normalizada para escala entre 0-1
    image_arr = np.array(image, dtype=np.float32) / 255.0

    # Quantizar o float32 para uint8
    if input_details["dtype"] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        image_arr = image_arr / input_scale + input_zero_point

    # Expandir a array em 1 dimensão
    image_arr = np.expand_dims(image_arr, axis=0).astype(input_details["dtype"])

    # Jogar para memoria do modelo
    interpreter.set_tensor(input_details["index"], image_arr)

    # Fazer inferencia da image
    interpreter.invoke()

    # Jogar em uma variavel o que tem na saida do modelo
    output = interpreter.get_tensor(output_details["index"])[0]

    display(output)

    # Retorna o valor de uint8 para flot32
    if output_details["dtype"] == np.uint8:
        output_scale, output_zero_point = output_details["quantization"]
        output = output_scale * (output.astype(np.float32) - output_zero_point)

    confidences = {LABELS[i]: float(confidence) for i, confidence in enumerate(output)}

    print(confidences, output.argmax())

    return confidences

#### Exemplo de Parâmetros de ENTRADA e SAÍDA do modelo

In [ ]:
input_details = interpreter.get_input_details()[0]
display(input_details)

In [ ]:
output_details = interpreter.get_output_details()[0]
display(output_details)

# 
---

### 6º PASSO: CRIAR A INTERFACE COM O GRADIO

Criação da interface que utiliza da função **_predict(image)_**

In [ ]:
gr.Interface(
    fn=predict,
    inputs=[gr.Image(type="pil")],
    outputs=[gr.Label()],
).launch()

####
---

#### 💡VALIDE O MODELO ATRAVÉS DA INTERFACE
- É possível testar o modelo pelo próprio _output_ da célula do notebook
- Outra opção é clicar no link gerado e testá-lo em seu _browser_ de preferência